<a href="https://colab.research.google.com/github/QuratulAin20/Deep-Learning/blob/main/Book_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import files

# Upload the book file
uploaded = files.upload()

# If you want to check the names of uploaded files
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')

Saving text_data.txt to text_data (1).txt
Uploaded file: text_data (1).txt


In [11]:

!pip install transformers
!pip install keras
!pip install nltk
!pip install streamlit
import pandas as pd
import numpy as np


## Step 2: Extract text from PDF

In [1]:
def extract_text_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# Specify the path to your .txt file
file_path = '/content/text_data.txt'
text = extract_text_from_txt(file_path)

In [14]:
text

'\nIntroduction\nNLTK is the Natural Language Toolkit, a comprehensive Python library for natural language \nprocessing and text analytics. Originally designed for teaching, it has been adopted in the \nindustry for research and development due to its usefulness and breadth of coverage.\nThis chapter will cover the basics of tokenizing text and using WordNet. Tokenization is a \nmethod of breaking up a piece of text into many pieces, and is an essential irst step for \nrecipes in later chapters.\nTokenizing Text and WordNet Basics\n\nWordNet is a dictionary designed for programmatic access by natural language processing \nsystems. NLTK includes a WordNet corpus reader, which we will use to access and explore \nWordNet. We\'ll be using WordNet again in later chapters, so it\'s important to familiarize \nyourself with the basics irst.\nTokenizing text into sentences\nTokenization is the process of splitting a string into a list of pieces, or tokens. We\'ll start by \nsplitting a paragrap

## Step 3:Text Preprocessing

In [3]:

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')


# Text preprocessing
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove numbers, punctuations, and special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenization
    tokens = word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    # Join tokens back to a string
    return ' '.join(tokens)

cleaned_text = preprocess_text(text)
print(cleaned_text[:500])  # Display first 500 characters of cleaned text


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


introduction nltk natural language toolkit comprehensive python library natural language processing text analytics originally designed teaching adopted industry research development due usefulness breadth coverage chapter cover basics tokenizing text using wordnet tokenization method breaking piece text many pieces essential irst step recipes later chapters tokenizing text wordnet basics wordnet dictionary designed programmatic access natural language processing systems nltk includes wordnet cor


# Step 4: Text Segmentation and Vectorization

In [4]:
import tensorflow as tf
# Tokenization and vectorization
tokenizer = Tokenizer()
tokenizer.fit_on_texts([cleaned_text])
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for i in range(1, len(cleaned_text.split())):
    n_gram_sequence = cleaned_text.split()[:i+1]
    token_list = tokenizer.texts_to_sequences([n_gram_sequence])[0]
    input_sequences.append(token_list)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Create predictors and label (Shifted sequence for LSTM prediction)
X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)


## Step5 Model Selection LSTM

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Model definition
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len - 1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X, y, epochs=10, verbose=1)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 22s 140ms/step - accuracy: 0.0145 - loss: 7.0156
Epoch 2/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 16s 133ms/step - accuracy: 0.0187 - loss: 6.5896
Epoch 3/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 21s 136ms/step - accuracy: 0.0268 - loss: 6.3870
Epoch 4/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 16s 134ms/step - accuracy: 0.0294 - loss: 6.2071
Epoch 5/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 17s 136ms/step - accuracy: 0.0373 - loss: 5.9584
Epoch 6/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 21s 138ms/step - accuracy: 0.0489 - loss: 5.6494
Epoch 7/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 17s 141ms/step - accuracy: 0.0680 - loss: 5.3075
Epoch 8/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 18s 144ms/step - accuracy: 0.0935 - loss: 4.9117
Epoch 9/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 20s 140ms/step - accuracy: 0.1255 - loss: 4.5747
Epoch 10/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 20s 140ms/step - accuracy: 0.1804 - loss: 4.1495


## Step-6: Building QA Functionality

In [11]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_answer(seed_text, next_words):
    for _ in range(next_words):
        # Convert the seed text to sequences
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        # Pad the sequences
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')

        # Predict the next word probabilities
        predicted_probs = model.predict(token_list, verbose=0)

        # Get the index of the highest probability
        predicted_index = np.argmax(predicted_probs, axis=-1)[0]

        # Find the word corresponding to the predicted index
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break

        # Append the predicted word to the seed text
        seed_text += " " + output_word

    return seed_text

# Example question
question = "What is the "
answer = generate_answer(question, 20)  # Generate 20 words as an answer
print("Answer:", answer)

Answer: What is the  language language language language language language language language language language language language language language language language language language language language


## Step 7: Build the User Interface (UI)

In [14]:
# Install Flask and Flask-Ngrok
!pip install flask-ngrok

from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
from threading import Thread

app = Flask(__name__)
run_with_ngrok(app)  # Starts the app on Colab

@app.route('/')
def home():
    return '''
        <html>
            <body>
                <h1>Chatbot</h1>
                <input id="user_input" type="text" placeholder="Type your message here...">
                <button onclick="sendMessage()">Send</button>
                <div id="response"></div>

                <script>
                    function sendMessage() {
                        var userText = document.getElementById("user_input").value;
                        fetch("/get?msg=" + userText)
                            .then(response => response.text())
                            .then(data => {
                                document.getElementById("response").innerText = data;
                            });
                    }
                </script>
            </body>
        </html>
    '''

@app.route('/get')
def get_bot_response():
    userText = request.args.get('msg')
    answer = generate_answer(userText, 10)  # Generate a response (ensure generate_answer is defined)
    return str(answer)
# Run the Flask app in a separate thread
def run():
    app.run()

Thread(target=run).start()



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
